# تحليل المشاعر باستخدام BERT

هذا المشروع يستخدم نموذج BERT لتحليل المشاعر في النصوص العربية والإنجليزية.

In [ ]:
# تحميل Google Drive في Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENV = True
    print('تم تحميل Google Drive بنجاح!')
except ImportError:
    COLAB_ENV = False
    print('البيئة المحلية - لا حاجة لتحميل Drive')

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install transformers datasets torch scikit-learn seqeval
!pip install --upgrade transformers

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# تحديد مسارات البيانات
if 'COLAB_ENV' in globals() and COLAB_ENV:
    # مسار البيانات في Google Drive
    DATA_PATH = '/content/drive/MyDrive/AI-Projects-main/db'
    MODEL_SAVE_PATH = '/content/drive/MyDrive/AI-Projects-main/models'
else:
    # مسار البيانات المحلي
    DATA_PATH = r'c:\\Users\\Administrator\\Desktop\\AI-Projects-main\\db'
    MODEL_SAVE_PATH = r'c:\\Users\\Administrator\\Desktop\\AI-Projects-main\\models'

# إنشاء مجلد النماذج إذا لم يكن موجوداً
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
print(f'مسار البيانات: {DATA_PATH}')
print(f'مسار حفظ النماذج: {MODEL_SAVE_PATH}')

## تحميل ومعالجة البيانات

سنقوم بتحميل البيانات من ملفات JSONL وتحويلها لتحليل المشاعر

In [ ]:
def load_jsonl_data(data_path):
    """تحميل البيانات من ملفات JSONL"""
    all_data = []
    
    # البحث عن جميع ملفات JSONL في المجلد
    jsonl_files = []
    for file in os.listdir(data_path):
        if file.endswith('.jsonl'):
            jsonl_files.append(os.path.join(data_path, file))
    
    print(f'تم العثور على {len(jsonl_files)} ملف JSONL')
    
    for file_path in jsonl_files:
        print(f'تحميل: {os.path.basename(file_path)}')
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f):
                try:
                    data = json.loads(line.strip())
                    if 'input' in data and 'response' in data:
                        all_data.append({
                            'text': data['input'],
                            'response': data['response']
                        })
                except json.JSONDecodeError as e:
                    print(f'خطأ في السطر {line_num + 1} من {os.path.basename(file_path)}: {e}')
                    continue
    
    print(f'تم تحميل {len(all_data)} عينة من البيانات')
    return all_data

In [ ]:
def analyze_sentiment_simple(text):
    """تحليل بسيط للمشاعر بناءً على الكلمات المفتاحية"""
    positive_words = ['جيد', 'ممتاز', 'رائع', 'أحب', 'مفيد', 'جميل', 'نجح', 'توفيق', 'شكرا', 'أشكرك']
    negative_words = ['سيء', 'فشل', 'مشكلة', 'خطأ', 'صعب', 'مستحيل', 'لا أحب', 'مزعج']
    
    text_lower = text.lower()
    
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        return 2  # positive
    elif negative_count > positive_count:
        return 0  # negative
    else:
        return 1  # neutral

def prepare_sentiment_data(raw_data, max_samples=10000):
    """تحضير البيانات لتدريب نموذج تحليل المشاعر"""
    processed_data = []
    
    # أخذ عينة من البيانات إذا كانت كبيرة
    if len(raw_data) > max_samples:
        raw_data = raw_data[:max_samples]
        print(f'تم أخذ عينة من {max_samples} عنصر من البيانات')
    
    for item in raw_data:
        text = item['text']
        response = item['response']
        
        # تحليل المشاعر للنص والرد
        input_sentiment = analyze_sentiment_simple(text)
        response_sentiment = analyze_sentiment_simple(response)
        
        # إضافة النص مع تصنيف المشاعر
        processed_data.append({
            'text': text,
            'label': input_sentiment
        })
        
        processed_data.append({
            'text': response,
            'label': response_sentiment
        })
    
    return processed_data

In [ ]:
# تحميل البيانات
print('بدء تحميل البيانات...')
raw_data = load_jsonl_data(DATA_PATH)

# تحضير البيانات لتحليل المشاعر
print('تحضير البيانات لتحليل المشاعر...')
sentiment_data = prepare_sentiment_data(raw_data, max_samples=20000)

# عرض إحصائيات البيانات
labels = [item['label'] for item in sentiment_data]
label_counts = pd.Series(labels).value_counts()
print('\nتوزيع التصنيفات:')
print(f'سلبي (0): {label_counts.get(0, 0)}')
print(f'محايد (1): {label_counts.get(1, 0)}')
print(f'إيجابي (2): {label_counts.get(2, 0)}')

# عرض عينات من البيانات
print('\nعينات من البيانات:')
for i in range(min(5, len(sentiment_data))):
    item = sentiment_data[i]
    label_name = ['سلبي', 'محايد', 'إيجابي'][item['label']]
    print(f'{i+1}. النص: {item["text"][:100]}...')
    print(f'   التصنيف: {label_name}\n')

In [ ]:
# إعداد المحلل اللغوي
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# تشفير البيانات
def encode_data(data_list):
    encoded_data = []
    
    for item in data_list:
        encoding = tokenizer(
            item['text'],
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        )
        
        encoded_item = {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'label': item['label']
        }
        encoded_data.append(encoded_item)
    
    return encoded_data

print('تشفير البيانات...')
encoded_sentiment_data = encode_data(sentiment_data)
print(f'تم تشفير {len(encoded_sentiment_data)} عينة')

## إعداد نموذج PyTorch

In [ ]:
# فئة البيانات لـ PyTorch
class SentimentDataset(Dataset):
    def __init__(self, data):
        self.data = data  # قائمة من القواميس

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': item['input_ids'],
            'attention_mask': item['attention_mask'],
            'labels': torch.tensor(item['label'], dtype=torch.long)
        }

In [ ]:
# تقسيم البيانات إلى تدريب وتقييم
train_data, val_data = train_test_split(encoded_sentiment_data, test_size=0.2, random_state=42)

train_dataset = SentimentDataset(train_data)
val_dataset = SentimentDataset(val_data)

print(f'بيانات التدريب: {len(train_dataset)} عينة')
print(f'بيانات التقييم: {len(val_dataset)} عينة')

In [ ]:
# تحميل نموذج BERT متعدد اللغات
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased', 
    num_labels=3
)

print('تم تحميل النموذج بنجاح!')

In [ ]:
# إعدادات التدريب
training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_PATH, 'sentiment_model'),
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    logging_dir=os.path.join(MODEL_SAVE_PATH, 'logs'),
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=2,
    report_to='none',  # تعطيل WandB/MLFlow
    dataloader_pin_memory=False,  # لتجنب مشاكل الذاكرة في Colab
)

print(f'مجلد حفظ النموذج: {training_args.output_dir}')

In [ ]:
def compute_metrics(eval_pred):
    """حساب مقاييس الأداء"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro')
    }

In [ ]:
# إنشاء المدرب وبدء التدريب
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print('بدء تدريب النموذج...')
trainer.train()

print('تم الانتهاء من التدريب!')

In [ ]:
def classify_sentiment(text):
    """تصنيف المشاعر للنص المدخل"""
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=128
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
        probabilities = torch.softmax(logits, dim=1)[0]
    
    label_map = {0: 'سلبي', 1: 'محايد', 2: 'إيجابي'}
    confidence = probabilities[prediction].item()
    
    return {
        'label': label_map[prediction],
        'confidence': confidence,
        'probabilities': {
            'سلبي': probabilities[0].item(),
            'محايد': probabilities[1].item(),
            'إيجابي': probabilities[2].item()
        }
    }

In [ ]:
# اختبار النموذج
test_texts = [
    'هذا المنتج رائع جداً وأنصح به',
    'المنتج سيء ولا أنصح بشرائه',
    'المنتج عادي، لا بأس به',
    'This product is amazing!',
    'I hate this product'
]

print('اختبار النموذج:')
for text in test_texts:
    result = classify_sentiment(text)
    print(f'النص: {text}')
    print(f'التصنيف: {result["label"]} (ثقة: {result["confidence"]:.2f})')
    print('---')

### استكمال التدريب

##### 1- من نقطة التوقف (نفس البيانات)

In [ ]:
# trainer.train(resume_from_checkpoint="path/to/checkpoint")

##### 2- تحميل النموذج (بيانات مختلفة)

In [ ]:
# model = BertForSequenceClassification.from_pretrained("path/to/checkpoint")

### النشر

In [ ]:
# حفظ النموذج للنشر
deployment_path = os.path.join(MODEL_SAVE_PATH, 'Sentiment_Model_Final')
model.save_pretrained(deployment_path)
tokenizer.save_pretrained(deployment_path)

print(f'تم حفظ النموذج في: {deployment_path}')
print('يمكن الآن استخدام النموذج للنشر!')